In [2]:
import os
import numpy as np
import pandas as pd
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
from sklearn.model_selection import train_test_split
import torch
import random
from tqdm import tqdm
from transformers import BertJapaneseTokenizer, BertForMaskedLM
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification,AdamW, BertConfig, BertModel, BertPreTrainedModel
import pytorch_lightning as pl
#MODEL_NAME='bert-base-cased'
from torch import nn
import MeCab
from sklearn.preprocessing import LabelEncoder
batch=2
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
os.environ["CUDA_VISIBLE_DEVICES"]="0"
tagger = MeCab.Tagger()
tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [5]:
df = pd.read_csv("/home/is/shuntaro-o/dev/persons_move_analysis/data/202271month_per_hour_geotaged_adGeocode_Tokyo.csv")
df_Tokyo = pd.read_csv('/home/is/shuntaro-o/dev/persons_move_analysis/data/use_new_model/202271month_per_hour_noGeo_adEstimate_Tokyo.csv')

In [7]:
y = "code"
L = LabelEncoder()
df[y] = L.fit_transform(df[y])
num_class = df[y].max() + 1  # オリジンの補正

In [9]:
from torch import nn
import torch.nn.functional as F
from transformers import AutoModel
from transformers import BertModel
class BertForSequenceClassifier_pl(pl.LightningModule):
        
    def __init__(self, model_name, lr, num_class):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()
        
        # 引数のnum_labelsとlrを保存。
        # 例えば、self.hparams.lrでlrにアクセスできる。
        # チェックポイント作成時にも自動で保存される。
        self.save_hyperparameters() 

        # BERTのロード
        self.bert = BertModel.from_pretrained(model_name)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_class)
        self.criterion = nn.CrossEntropyLoss()

        # BertLayerモジュールの最後を勾配計算ありに変更
        for param in self.bert.parameters():
            param.requires_grad = False
        for param in self.bert.encoder.layer[-1].parameters():
            param.requires_grad = True

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.bert(input_ids, attention_mask=attention_mask)
        preds= self.classifier(output.pooler_output)
        
        loss = 0
        if labels is not None:
            loss = self.criterion(preds, labels)
        #print(f"tihi is {loss}")
        return loss, preds
    
       # trainのミニバッチに対して行う処理
    def training_step(self, batch, batch_idx):
        loss, preds = self.forward(input_ids=batch["input_ids"],
                                    attention_mask=batch["attention_mask"],
                                    labels=batch["labels"])
        self.log('train_loss', loss)
        return {'loss': loss,
                'batch_preds': preds,
                'batch_labels': batch["labels"]}

    # validation、testでもtrain_stepと同じ処理を行う
    def validation_step(self, batch, batch_idx):
        loss, preds = self.forward(input_ids=batch["input_ids"],
                                    attention_mask=batch["attention_mask"],
                                    labels=batch["labels"])
        return {'loss': loss,
                'batch_preds': preds,
                'batch_labels': batch["labels"]}

    def test_step(self, batch, batch_idx):
        loss, preds = self.forward(input_ids=batch["input_ids"],
                                    attention_mask=batch["attention_mask"],
                                    labels=batch["labels"])
        return {'loss': loss,
                'batch_preds': preds,
                'batch_labels': batch["labels"]}

    # epoch終了時にvalidationのlossとaccuracyを記録
    def validation_epoch_end(self, outputs, mode="val"):
        # loss計算
        epoch_preds = torch.cat([x['batch_preds'] for x in outputs])
        epoch_labels = torch.cat([x['batch_labels'] for x in outputs])
        epoch_loss = self.criterion(epoch_preds, epoch_labels)
        self.log(f"{mode}_loss", epoch_loss, logger=True)

        num_correct = (epoch_preds.argmax(dim=1) == epoch_labels).sum().item()
        epoch_accuracy = num_correct / len(epoch_labels)
        self.log(f"{mode}_accuracy", epoch_accuracy, logger=True)

        

    # testデータのlossとaccuracyを算出（validationの使いまわし）
    def test_epoch_end(self, outputs):
        return self.validation_epoch_end(outputs, "test")

    # 学習に用いるオプティマイザを返す関数を書く。
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [10]:
best_model_path = '/home/is/shuntaro-o/dev/persons_move_analysis/models/Tokyo_model_ver2.ckpt'

In [11]:
x = "text"
df_Tokyo[x] = df_Tokyo[x].astype(str)
df_Tokyo[x]=df_Tokyo[x].astype(str)
df_Tokyo[x] = df_Tokyo[x].str.replace(r'[【】]', ' ')       # 【】の除去
df_Tokyo[x] = df_Tokyo[x].str.replace(r'[（）()]', ' ')     # （）の除去
df_Tokyo[x] = df_Tokyo[x].str.replace(r'[［］\[\]]', ' ')   # ［］の除去
df_Tokyo[x] = df_Tokyo[x].str.replace(r'[@＠]\w+', '')  # メンションの除去
df_Tokyo[x]= df_Tokyo[x].str.replace(r'_', '')#underscodf_Tokyo[x].str 
df_Tokyo[x] = df_Tokyo[x].str.replace(r'https?:\/\/.*?[\r\n ]', '')  # URLの除去
df_Tokyo[x] = df_Tokyo[x].str.replace(r'http:\/\/.*?[\r\n ]', '')  # URLの除去
df_Tokyo[x] = df_Tokyo[x].str.replace(r'　', ' ')

/tmp/ipykernel_47732/2582238918.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_Tokyo[x] = df_Tokyo[x].str.replace(r'[【】]', ' ')       # 【】の除去
/tmp/ipykernel_47732/2582238918.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_Tokyo[x] = df_Tokyo[x].str.replace(r'[（）()]', ' ')     # （）の除去
/tmp/ipykernel_47732/2582238918.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df_Tokyo[x] = df_Tokyo[x].str.replace(r'[［］\[\]]', ' ')   # ［］の除去
/tmp/ipykernel_47732/2582238918.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df_Tokyo[x] = df_Tokyo[x].str.replace(r'[@＠]\w+', '')  # メンションの除去
/tmp/ipykernel_47732/2582238918.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df_Tokyo[x] = df_Tokyo[x].str.replace(r'https?:\/\/.*?[\r\n ]', ''

In [12]:
sentences_text_test=df_Tokyo.text.values
list_estimate = []
model = BertForSequenceClassifier_pl.load_from_checkpoint(best_model_path)
bert=model.bert.cuda()
classifier=model.classifier.cuda()
for sentence in sentences_text_test:
    encoding = tokenizer(
    sentence,
    padding = 'longest',
    return_tensors='pt')
    encoding = { k: v.cuda() for k, v in encoding.items() }
    with torch.no_grad():
        output = bert(**encoding)
        ans=classifier(output.pooler_output)
        ans = ans.to('cpu').detach().numpy().copy()
        ans = ans.argmax()
        ans = L.inverse_transform([ans])[0]
        list_estimate.append(ans)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
list_estimate=pd.Series(list_estimate)
df_Tokyo = df_Tokyo.reset_index(drop=True)
df_Tokyo_adEstimate=pd.concat([df_Tokyo,list_estimate],axis=1)
df_Tokyo_adEstimate=df_Tokyo_adEstimate.rename(columns={0: 'Tokyo_code'})

In [ ]:
df_Tokyo_adEstimate.to_csv('/home/is/shuntaro-o/dev/persons_move_analysis/data/use_new_model/202271month_per_hour_noGeo_adEstimate_Tokyo_ad2zimethestimate.csv')